# Challenge 2: Sentiment Analysis

In this challenge we will learn sentiment analysis and practice performing sentiment analysis on Twitter tweets. 

## Introduction

Sentiment analysis is to *systematically identify, extract, quantify, and study affective states and subjective information* based on texts ([reference](https://en.wikipedia.org/wiki/Sentiment_analysis)). In simple words, it's to understand whether a person is happy or unhappy in producing the piece of text. Why we (or rather, companies) care about sentiment in texts? It's because by understanding the sentiments in texts, we will be able to know if our customers are happy or unhappy about our products and services. If they are unhappy, the subsequent action is to figure out what have caused the unhappiness and make improvements.

Basic sentiment analysis only understands the *positive* or *negative* (sometimes *neutral* too) polarities of the sentiment. More advanced sentiment analysis will also consider dimensions such as agreement, subjectivity, confidence, irony, and so on. In this challenge we will conduct the basic positive vs negative sentiment analysis based on real Twitter tweets.

NLTK comes with a [sentiment analysis package](https://www.nltk.org/api/nltk.sentiment.html). This package is great for dummies to perform sentiment analysis because it requires only the textual data to make predictions. For example:

```python
>>> from nltk.sentiment.vader import SentimentIntensityAnalyzer
>>> txt = "Ironhack is a Global Tech School ranked num 2 worldwide.   Our mission is to help people transform their careers and join a thriving community of tech professionals that love what they do."
>>> analyzer = SentimentIntensityAnalyzer()
>>> analyzer.polarity_scores(txt)
{'neg': 0.0, 'neu': 0.741, 'pos': 0.259, 'compound': 0.8442}
```

In this challenge, however, you will not use NLTK's sentiment analysis package because in your Machine Learning training in the past 2 weeks you have learned how to make predictions more accurate than that. The [tweets data](https://www.kaggle.com/kazanova/sentiment140) we will be using today are already coded for the positive/negative sentiment. You will be able to use the Naïve Bayes classifier you learned in the lesson to predict the sentiment of tweets based on the labels.

## Conducting Sentiment Analysis

### Loading and Exploring Data

The dataset we'll be using today is located on Kaggle (https://www.kaggle.com/kazanova/sentiment140). Once you have downloaded and imported the dataset, it you will need to define the columns names: df.columns = ['target','id','date','flag','user','text']

*Notes:* 

* The dataset is huuuuge (1.6m tweets). When you develop your data analysis codes, you can sample a subset of the data (e.g. 20k records) so that you will save a lot of time when you test your codes.

In [1]:
import pandas as pd
import numpy as np
from random import sample
from sklearn.feature_extraction.text import CountVectorizer
from nltk.probability import FreqDist

In [2]:
data = pd.read_csv('Sentiment140.csv')
data

,target,ids,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
...,...,...,...,...,...,...
1599995,4,2193601966,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,AmandaMarie1028,Just woke up. Having no school is the best fee...
1599996,4,2193601969,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,TheWDBoards,TheWDB.com - Very cool to hear old Walt interv...
1599997,4,2193601991,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,bpbabe,Are you ready for your MoJo Makeover? Ask me f...
1599998,4,2193602064,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,tinydiamondz,Happy 38th Birthday to my boo of alll time!!! ...


In [3]:
#sample subset for inital testing
data_sample=data.sample(n=7000, random_state=22)
data_sample

,target,ids,date,flag,user,text
1013875,4,1881370700,Fri May 22 04:27:56 PDT 2009,NO_QUERY,FreshPlastic,#followfriday @foyboy 1) She had Skips on her ...
1064419,4,1964672525,Fri May 29 14:36:52 PDT 2009,NO_QUERY,CarebearJK,@PaulaAbdul we are finally starting to see the...
290763,0,1995464536,Mon Jun 01 13:31:56 PDT 2009,NO_QUERY,jessicabalicki,w. all my old coworkers going to the wake RIP...
770144,0,2301890082,Tue Jun 23 16:03:01 PDT 2009,NO_QUERY,dlphntat,"yeah, Bravo TV. I so wanna learn more about th..."
1266526,4,1999668574,Mon Jun 01 20:36:58 PDT 2009,NO_QUERY,stevefroisland,"@JCTrick Thanks, J! I think I'm almost getting..."
...,...,...,...,...,...,...
1206295,4,1986765579,Sun May 31 19:17:15 PDT 2009,NO_QUERY,MiizLushious,@iHuntMidgets &amp; take me for long walks on ...
268648,0,1989447691,Mon Jun 01 01:34:38 PDT 2009,NO_QUERY,xnix,"... a free day from work and it's rainy, yay"
124307,0,1834086655,Mon May 18 01:34:39 PDT 2009,NO_QUERY,Clair12,"I think it's about time I get up, make lunch a..."
912978,4,1752473298,Sat May 09 22:01:20 PDT 2009,NO_QUERY,texas_girl_90,Tryin to figure all this out!! LOL


### Prepare Textual Data for Sentiment Analysis

Now, apply the functions you have written in Challenge 1 to your whole data set. These functions include:

* `clean_up()`

* `tokenize()`

* `stem_and_lemmatize()`

* `remove_stopwords()`

Create a new column called `text_processed` in the dataframe to contain the processed data. At the end, your `text_processed` column should contain lists of word tokens that are cleaned up. Your data should look like below:

![Processed Data](data-cleaning-results.png)

In [4]:
#import functions from other notebook in the directory, also imports libaries needed for the functions
from challenge_1_functions import clean_up, tokenize, stem_and_lemmatize, remove_stopwords

cleaned=data_sample['text'].apply(clean_up)
tok=cleaned.apply(tokenize)
s_and_l=tok.apply(stem_and_lemmatize)
processed=s_and_l.apply(remove_stopwords)
processed


[nltk_data] Downloading package punkt to /Users/test/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/test/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /Users/test/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


1013875    [followfriday, foyboy, skip, earlier, week, ch...
1064419    [paulaabdul, final, start, see, sun, warm, wea...
290763         [w, old, cowork, go, wake, rip, michel, xoxo]
770144     [yeah, bravo, tv, wan, learn, real, housew, or...
1266526           [jctrick, thank, think, almost, get, hang]
                                 ...                        
1206295                [ihuntmidget, amp, long, walk, beach]
268648                         [free, day, work, raini, yay]
124307     [think, time, get, make, lunch, strat, get, re...
912978                                   [tryin, figur, lol]
46010      [littl, china, ducki, dish, sinc, wa, babi, fe...
Name: text, Length: 7000, dtype: object

In [5]:
data_sample['text_processed']=processed
data_sample

,target,ids,date,flag,user,text,text_processed
1013875,4,1881370700,Fri May 22 04:27:56 PDT 2009,NO_QUERY,FreshPlastic,#followfriday @foyboy 1) She had Skips on her ...,"[followfriday, foyboy, skip, earlier, week, ch..."
1064419,4,1964672525,Fri May 29 14:36:52 PDT 2009,NO_QUERY,CarebearJK,@PaulaAbdul we are finally starting to see the...,"[paulaabdul, final, start, see, sun, warm, wea..."
290763,0,1995464536,Mon Jun 01 13:31:56 PDT 2009,NO_QUERY,jessicabalicki,w. all my old coworkers going to the wake RIP...,"[w, old, cowork, go, wake, rip, michel, xoxo]"
770144,0,2301890082,Tue Jun 23 16:03:01 PDT 2009,NO_QUERY,dlphntat,"yeah, Bravo TV. I so wanna learn more about th...","[yeah, bravo, tv, wan, learn, real, housew, or..."
1266526,4,1999668574,Mon Jun 01 20:36:58 PDT 2009,NO_QUERY,stevefroisland,"@JCTrick Thanks, J! I think I'm almost getting...","[jctrick, thank, think, almost, get, hang]"
...,...,...,...,...,...,...,...
1206295,4,1986765579,Sun May 31 19:17:15 PDT 2009,NO_QUERY,MiizLushious,@iHuntMidgets &amp; take me for long walks on ...,"[ihuntmidget, amp, long, walk, beach]"
268648,0,1989447691,Mon Jun 01 01:34:38 PDT 2009,NO_QUERY,xnix,"... a free day from work and it's rainy, yay","[free, day, work, raini, yay]"
124307,0,1834086655,Mon May 18 01:34:39 PDT 2009,NO_QUERY,Clair12,"I think it's about time I get up, make lunch a...","[think, time, get, make, lunch, strat, get, re..."
912978,4,1752473298,Sat May 09 22:01:20 PDT 2009,NO_QUERY,texas_girl_90,Tryin to figure all this out!! LOL,"[tryin, figur, lol]"


### Creating Bag of Words

The purpose of this step is to create a [bag of words](https://en.wikipedia.org/wiki/Bag-of-words_model) from the processed data. The bag of words contains all the unique words in your whole text body (a.k.a. *corpus*) with the number of occurrence of each word. It will allow you to understand which words are the most important features across the whole corpus.

Also, you can imagine you will have a massive set of words. The less important words (i.e. those of very low number of occurrence) do not contribute much to the sentiment. Therefore, you only need to use the most important words to build your feature set in the next step. In our case, we will use the top 5,000 words with the highest frequency to build the features.

In the cell below, combine all the words in `text_processed` and calculate the frequency distribution of all words. A convenient library to calculate the term frequency distribution is NLTK's `FreqDist` class ([documentation](https://www.nltk.org/api/nltk.html#module-nltk.probability)). Then select the top 5,000 words from the frequency distribution.

In [6]:
word_list=[]

for i in range(data_sample.shape[0]):
    T=data_sample.text_processed.iloc[i]#+data_sample.text_processed.iloc[2]
    word_list=word_list+T


In [7]:
#got one list with all the words inside
len(word_list)

50554

In [8]:
FD=FreqDist(word_list)
FD

FreqDist({'go': 624, 'get': 487, 'wa': 479, 'day': 458, 'work': 409, 'good': 404, 'quot': 352, 'love': 346, 'like': 345, 'got': 312, ...})

In [9]:
# use top 5000 words, we dont need the values of only words
top_words=list(FD)[0:5000]
top_words

['go',
 'get',
 'wa',
 'day',
 'work',
 'good',
 'quot',
 'love',
 'like',
 'got',
 'thank',
 'today',
 'time',
 'lol',
 'think',
 'back',
 'see',
 'miss',
 'feel',
 'know',
 'need',
 'realli',
 'amp',
 'watch',
 'well',
 'make',
 'new',
 'hope',
 'still',
 'oh',
 'night',
 'look',
 'home',
 'wish',
 'twitter',
 'morn',
 'great',
 'tomorrow',
 'much',
 'sleep',
 'bad',
 'wait',
 'haha',
 'would',
 'http',
 'last',
 'happi',
 'sad',
 'follow',
 'nice',
 'say',
 'way',
 'whi',
 'right',
 'thing',
 'sorri',
 'onli',
 'week',
 'friend',
 'fun',
 'peopl',
 'even',
 'tweet',
 'though',
 'bed',
 'veri',
 'awesom',
 'dont',
 'tonight',
 'hate',
 'start',
 'could',
 'better',
 'next',
 'cant',
 'hi',
 'suck',
 'gon',
 'lt',
 'yeah',
 'ye',
 'soon',
 'school',
 'weekend',
 'hour',
 'play',
 'let',
 'hey',
 'plea',
 'rain',
 'guy',
 'readi',
 'call',
 'life',
 'show',
 'phone',
 'first',
 'never',
 'someon',
 'hous',
 'bore',
 'becaus',
 'head',
 'yet',
 'use',
 'sick',
 'final',
 'done',
 'long'

### Building Features

Now let's build the features. Using the top 5,000 words, create a 2-dimensional matrix to record whether each of those words is contained in each document (tweet). Then you also have an output column to indicate whether the sentiment in each tweet is positive. For example, assuming your bag of words has 5 items (`['one', 'two', 'three', 'four', 'five']`) out of 4 documents (`['A', 'B', 'C', 'D']`), your feature set is essentially:

| Doc | one | two | three | four | five | is_positive |
|---|---|---|---|---|---|---|
| A | True | False | False | True | False | True |
| B | False | False | False | True | True | False |
| C | False | True | False | False | False | True |
| D | True | False | False | False | True | False|

However, because the `nltk.NaiveBayesClassifier.train` class we will use in the next step does not work with Pandas dataframe, the structure of your feature set should be converted to the Python list looking like below:

```python
[
	({
		'one': True,
		'two': False,
		'three': False,
		'four': True,
		'five': False
	}, True),
	({
		'one': False,
		'two': False,
		'three': False,
		'four': True,
		'five': True
	}, False),
	({
		'one': False,
		'two': True,
		'three': False,
		'four': False,
		'five': False
	}, True),
	({
		'one': True,
		'two': False,
		'three': False,
		'four': False,
		'five': True
	}, False)
]
```

To help you in this step, watch the [following video](https://www.youtube.com/watch?v=-vVskDsHcVc) to learn how to build the feature set with Python and NLTK. The source code in this video can be found [here](https://pythonprogramming.net/words-as-features-nltk-tutorial/).

[![Building Features](building-features.jpg)](https://www.youtube.com/watch?v=-vVskDsHcVc)

In [10]:
# modify traget column (positve tweet or not into a boolnean, currently its zero or 4)
data_sample['is_positive']=data_sample.target.copy()
data_sample['is_positive']=data_sample.target.replace([4],1)
data_sample['is_positive'] = data_sample['is_positive'].astype('bool')

In [11]:
data_sample.head(1)

,target,ids,date,flag,user,text,text_processed,is_positive
1013875,4,1881370700,Fri May 22 04:27:56 PDT 2009,NO_QUERY,FreshPlastic,#followfriday @foyboy 1) She had Skips on her ...,"[followfriday, foyboy, skip, earlier, week, ch...",True


In [12]:
#check if each of the top words in contained in each tweet or not

#words=data_sample.text_processed.iloc[0]

def get_features(words):
    features={}
    for w in top_words:
        features[w]= (w in words)
    
    return features 


In [13]:
#this is for one tweet as a test
get_features(data_sample.text_processed.iloc[1])

{'go': False,
 'get': False,
 'wa': False,
 'day': False,
 'work': False,
 'good': False,
 'quot': False,
 'love': False,
 'like': False,
 'got': False,
 'thank': False,
 'today': False,
 'time': False,
 'lol': False,
 'think': False,
 'back': False,
 'see': True,
 'miss': False,
 'feel': False,
 'know': False,
 'need': False,
 'realli': False,
 'amp': False,
 'watch': False,
 'well': False,
 'make': False,
 'new': False,
 'hope': False,
 'still': False,
 'oh': False,
 'night': False,
 'look': False,
 'home': False,
 'wish': False,
 'twitter': False,
 'morn': False,
 'great': False,
 'tomorrow': False,
 'much': False,
 'sleep': False,
 'bad': False,
 'wait': False,
 'haha': False,
 'would': False,
 'http': False,
 'last': False,
 'happi': False,
 'sad': False,
 'follow': False,
 'nice': False,
 'say': False,
 'way': False,
 'whi': False,
 'right': False,
 'thing': False,
 'sorri': False,
 'onli': False,
 'week': False,
 'friend': False,
 'fun': False,
 'peopl': False,
 'even': False,
 

In [14]:
#features for all tweets
features=[(get_features(data_sample['text_processed'].iloc[x]),data_sample['is_positive'].iloc[x]) for x in range(data_sample.shape[0])]

In [15]:
#look at result
features[1]

({'go': False,
  'get': False,
  'wa': False,
  'day': False,
  'work': False,
  'good': False,
  'quot': False,
  'love': False,
  'like': False,
  'got': False,
  'thank': False,
  'today': False,
  'time': False,
  'lol': False,
  'think': False,
  'back': False,
  'see': True,
  'miss': False,
  'feel': False,
  'know': False,
  'need': False,
  'realli': False,
  'amp': False,
  'watch': False,
  'well': False,
  'make': False,
  'new': False,
  'hope': False,
  'still': False,
  'oh': False,
  'night': False,
  'look': False,
  'home': False,
  'wish': False,
  'twitter': False,
  'morn': False,
  'great': False,
  'tomorrow': False,
  'much': False,
  'sleep': False,
  'bad': False,
  'wait': False,
  'haha': False,
  'would': False,
  'http': False,
  'last': False,
  'happi': False,
  'sad': False,
  'follow': False,
  'nice': False,
  'say': False,
  'way': False,
  'whi': False,
  'right': False,
  'thing': False,
  'sorri': False,
  'onli': False,
  'week': False,
  'friend

### Building and Traininng Naive Bayes Model

In this step you will split your feature set into a training and a test set. Then you will create a Bayes classifier instance using `nltk.NaiveBayesClassifier.train` ([example](https://www.nltk.org/book/ch06.html)) to train with the training dataset.

After training the model, call `classifier.show_most_informative_features()` to inspect the most important features. The output will look like:

```
Most Informative Features
	    snow = True            False : True   =     34.3 : 1.0
	  easter = True            False : True   =     26.2 : 1.0
	 headach = True            False : True   =     20.9 : 1.0
	    argh = True            False : True   =     17.6 : 1.0
	unfortun = True            False : True   =     16.9 : 1.0
	    jona = True             True : False  =     16.2 : 1.0
	     ach = True            False : True   =     14.9 : 1.0
	     sad = True            False : True   =     13.0 : 1.0
	  parent = True            False : True   =     12.9 : 1.0
	  spring = True            False : True   =     12.7 : 1.0
```

The [following video](https://www.youtube.com/watch?v=rISOsUaTrO4) will help you complete this step. The source code in this video can be found [here](https://pythonprogramming.net/naive-bayes-classifier-nltk-tutorial/).

[![Building and Training NB](nb-model-building.jpg)](https://www.youtube.com/watch?v=rISOsUaTrO4)

In [16]:
len(features)

7000

In [20]:
int((len(features)/100*70))

4900

In [23]:
#train-test-split

#sample is for now just 1000, so train with 70% and test with 30%
train_set=features[0:int((len(features)/100*70))]

test_set=features[int((len(features)/100*70)):]

In [25]:
# train classifier
import nltk 
classifier = nltk.NaiveBayesClassifier.train(train_set)

In [27]:
classifier.show_most_informative_features()

#ratio towards postive, so appear more often in positve tweets
#same for negative

Most Informative Features
                     sad = True            False : True   =     32.9 : 1.0
                    hurt = True            False : True   =     25.0 : 1.0
                  anymor = True            False : True   =     12.0 : 1.0
                     cri = True            False : True   =     12.0 : 1.0
                    poor = True            False : True   =     12.0 : 1.0
                   thank = True             True : False  =      8.3 : 1.0
                    gone = True            False : True   =      8.0 : 1.0
                 instead = True            False : True   =      7.5 : 1.0
                    beer = True             True : False  =      7.2 : 1.0
                    lone = True            False : True   =      6.8 : 1.0


### Testing Naive Bayes Model

Now we'll test our classifier with the test dataset. This is done by calling `nltk.classify.accuracy(classifier, test)`.

As mentioned in one of the tutorial videos, a Naive Bayes model is considered OK if your accuracy score is over 0.6. If your accuracy score is over 0.7, you've done a great job!

In [28]:
nltk.classify.accuracy(classifier, test_set)

0.6933333333333334

In [ ]:
#the test set doesnt have a accuracy over 0.7, but its getting very close

## Bonus Question 1: Improve Model Performance

If you are still not exhausted so far and want to dig deeper, try to improve your classifier performance. There are many aspects you can dig into, for example:

* Improve stemming and lemmatization. Inspect your bag of words and the most important features. Are there any words you should furuther remove from analysis? You can append these words to further remove to the stop words list.

* Remember we only used the top 5,000 features to build model? Try using different numbers of top features. The bottom line is to use as few features as you can without compromising your model performance. The fewer features you select into your model, the faster your model is trained. Then you can use a larger sample size to improve your model accuracy score.

In [ ]:
# your code here

## Bonus Question 2: Machine Learning Pipeline

In a new Jupyter Notebook, combine all your codes into a function (or a class). Your new function will execute the complete machine learning pipeline job by receiving the dataset location and output the classifier. This will allow you to use your function to predict the sentiment of any tweet in real time. 

In [ ]:
# your code here

## Bonus Question 3: Apache Spark

If you have completed the Apache Spark advanced topic lab, what you can do is to migrate your pipeline from local to a Databricks Notebook. Share your notebook with your instructor and classmates to show off your achievements!

In [ ]:
# your code here